# Data type sizes - a western myth

## Test Latin character strings with Latin collation

**Note:** My server default is SQL_Latin1_General_CP1_CI_AS

Set size limit of data types to be the same under Basic Multilingual Plane (BMP)
Characters between 1-byte (ASCII) and 3-bytes (East Asian)

In [15]:
DROP TABLE IF EXISTS t1;
CREATE TABLE t1 (c1 varchar(24) COLLATE Latin1_General_100_CI_AI, 
	c2 nvarchar(8) COLLATE Latin1_General_100_CI_AI);  
INSERT INTO t1 VALUES (N'MyString', N'MyString')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t1;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t1;
GO

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.068

varchar LEN,varchar DATALENGTH,c1
8,8,MyString


nvarchar LEN,nvarchar DATALENGTH,c2
8,16,MyString


That's as expected. So what was I talking about?

# Test Chinese character strings with Latin collation

In [16]:
DROP TABLE IF EXISTS t1;
CREATE TABLE t1 (c1 varchar(24) COLLATE Latin1_General_100_CI_AI, 
	c2 nvarchar(8) COLLATE Latin1_General_100_CI_AI);  
INSERT INTO t1 VALUES (N'敏捷的棕色狐狸跳', N'敏捷的棕色狐狸跳')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t1;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t1;
GO

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.031

varchar LEN,varchar DATALENGTH,c1
8,8,????????


nvarchar LEN,nvarchar DATALENGTH,c2
8,16,敏捷的棕色狐狸跳


uh-oh data loss on the varchar example. Why?

varchar is bound to code page enconding, and these code points cannot be found in the Latin code page.

In [3]:
SELECT ASCII('敏' COLLATE Latin1_General_100_CI_AI), CHAR(63)
SELECT ASCII('捷' COLLATE Latin1_General_100_CI_AI), CHAR(63)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.046

(No column name),(No column name)
63,?


(No column name),(No column name)
63,?


But why didn't it happen in the nvarchar example?

These Chinese characters are double-byte and within the *Basic Multilingual Plane* (BMP)

nvarchar with this non-SC collation encodes in UCS-2 (BMP), not the code page

In [5]:
SELECT UNICODE(N'敏' COLLATE Latin1_General_100_CI_AI), NCHAR(25935)
SELECT UNICODE(N'捷' COLLATE Latin1_General_100_CI_AI), NCHAR(25463)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.021

(No column name),(No column name)
25935,敏


(No column name),(No column name)
25463,捷


Irrespective of collation now. With a Unicode capable data type, collation only sets linguistic algorithms (Compare = sort; Case sensitivity = Upper/Lowercase)

In [9]:
SELECT UNICODE(N'敏' COLLATE Chinese_PRC_90_CI_AI), NCHAR(25935)
SELECT UNICODE(N'捷' COLLATE Chinese_PRC_90_CI_AI), NCHAR(25463)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.039

(No column name),(No column name)
25935,敏


(No column name),(No column name)
25463,捷


## Now test Chinese character strings with Chinese collation

In [17]:
DROP TABLE IF EXISTS t2;
CREATE TABLE t2 (c1 varchar(24) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI, 
	c2 nvarchar(8) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI);  
INSERT INTO t2 VALUES (N'敏捷的棕色狐狸跳', N'敏捷的棕色狐狸跳')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t2;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t2;
GO

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.037

varchar LEN,varchar DATALENGTH,c1
8,16,敏捷的棕色狐狸跳


nvarchar LEN,nvarchar DATALENGTH,c2
8,16,敏捷的棕色狐狸跳


Now the varchar example is correct. But there's 2 bytes per character?...

**Myth buster:** code page defines string length for varchar. It's not always 1 byte per character. 

Wasn't East-Asian 3 bytes? Yes, on UTF-8, but under Chinese collation code page, they are encoded using 2 bytes just like UCS-2/UTF-16


## Test with Supplementary Characters (4 bytes)

In [20]:
DROP TABLE IF EXISTS t2;
CREATE TABLE t2 (c1 varchar(24) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC, 
	c2 nvarchar(8) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC);  
INSERT INTO t2 VALUES (N'👶👦👧👨👩👴👵👨', N'👶👦👧👨👩👴👵👨')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t2;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t2;
GO

: Msg 2628, Level 16, State 1, Line 4
String or binary data would be truncated in table 'master.dbo.t2', column 'c2'. Truncated value: '👶👦👧👨'.

The statement has been terminated.

(0 rows affected)

(0 rows affected)

Total execution time: 00:00:00.038

varchar LEN,varchar DATALENGTH,c1


nvarchar LEN,nvarchar DATALENGTH,c2


uh-oh, let's set the proper data type length

In [21]:
DROP TABLE IF EXISTS t2;
CREATE TABLE t2 (c1 varchar(24) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC, 
	c2 nvarchar(16) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC);  
INSERT INTO t2 VALUES (N'👶👦👧👨👩👴👵👨', N'👶👦👧👨👩👴👵👨')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t2;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t2;
GO

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.029

varchar LEN,varchar DATALENGTH,c1
16,16,????????????????


nvarchar LEN,nvarchar DATALENGTH,c2
8,32,👶👦👧👨👩👴👵👨



Varchar still doesn't encode? 

In [22]:
DROP TABLE IF EXISTS t2;
CREATE TABLE t2 (c1 varchar(48) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC_UTF8, 
	c2 nvarchar(16) COLLATE Chinese_Traditional_Stroke_Order_100_CI_AI_SC);  
INSERT INTO t2 VALUES (N'👶👦👧👨👩👴👵👨', N'👶👦👧👨👩👴👵👨')  
SELECT LEN(c1) AS [varchar LEN],  
	DATALENGTH(c1) AS [varchar DATALENGTH], c1
FROM t2;  
SELECT LEN(c2) AS [nvarchar LEN], 
	DATALENGTH(c2) AS [nvarchar DATALENGTH], c2 
FROM t2;
GO

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.071

varchar LEN,varchar DATALENGTH,c1
8,32,👶👦👧👨👩👴👵👨


nvarchar LEN,nvarchar DATALENGTH,c2
8,32,👶👦👧👨👩👴👵👨


Finally!

What if I needed all these in one database? Easy, I could just use nvarchar.

In [23]:
DROP TABLE IF EXISTS t3;
CREATE TABLE t3 (c1 nvarchar(110) COLLATE Latin1_General_100_CI_AI_SC);  
INSERT INTO t3 VALUES (N'MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii敏捷的棕色狐狸跳👶👦')  
SELECT LEN(c1) AS [nvarchar UTF16 LEN],  
	DATALENGTH(c1) AS [nvarchar UTF16 DATALENGTH], c1
FROM t3; 
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.086

nvarchar UTF16 LEN,nvarchar UTF16 DATALENGTH,c1
65,134,MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii敏捷的棕色狐狸跳👶👦


But the majority of my data is set to Latin (ASCII)

In [24]:
DROP TABLE IF EXISTS t4;
CREATE TABLE t4 (c1 varchar(110) COLLATE Latin1_General_100_CI_AI_SC);  
INSERT INTO t4 VALUES (N'MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii敏捷的棕色狐狸跳👶👦')  
SELECT LEN(c1) AS [varchar UTF16 LEN],  
	DATALENGTH(c1) AS [varchar UTF16 DATALENGTH], c1
FROM t4; 
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.013

varchar UTF16 LEN,varchar UTF16 DATALENGTH,c1
67,67,MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii????????????


Where are the savings?

In [25]:
SELECT DATALENGTH(N'MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii') AS [Latin_UTF16_2bytes], 
	DATALENGTH(N'敏捷的棕色狐狸跳') AS [Chinese_UTF16_2bytes], 
	DATALENGTH(N'👶👦') AS [SC_UTF16_4bytes]
SELECT DATALENGTH('MyStringThequickbrownfoxjumpsoverthelazydogIsLatinAscii' COLLATE Latin1_General_100_CI_AI_SC_UTF8) AS [Latin_UTF8_1byte], 
	DATALENGTH('敏捷的棕色狐狸跳' COLLATE Latin1_General_100_CI_AI_SC_UTF8) AS [Chinese_UTF8_3bytes], 
	DATALENGTH('👶👦' COLLATE Latin1_General_100_CI_AI_SC_UTF8) AS [SC_UTF8_4bytes]
GO

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.043

Latin_UTF16_2bytes,Chinese_UTF16_2bytes,SC_UTF16_4bytes
110,16,8


Latin_UTF8_1byte,Chinese_UTF8_3bytes,SC_UTF8_4bytes
55,8,4
